In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm
from catboost import CatBoostRegressor
import os

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
!ls data_inner/nhanes_1994-2018/

demographic_path = 'data_inner/nhanes_1994-2018/demographics_clean.csv'
chemical_path = 'data_inner/nhanes_1994-2018/chemicals_clean.csv'
response_path = 'data_inner/nhanes_1994-2018/response_clean.csv'
dictionary_path = 'data_inner/nhanes_1994-2018/dictionary_nhanes.csv'
translations_path = 'data/columns_translations.csv'

# Load dataframes with specified columns
demographics = pd.read_csv(demographic_path, index_col=0, usecols=['RIDAGEYR', 'RIAGENDR', 'SEQN', 'SEQN_new'])
chemicals = pd.read_csv(chemical_path)
response = pd.read_csv(response_path)
translations = pd.read_csv(translations_path, index_col=0)

# Merge dataframes
df = demographics.merge(chemicals, how='inner', on=['SEQN', 'SEQN_new'])
df = df.merge(response, how='inner', on=['SEQN', 'SEQN_new'])

# Remove data with too many NaNs
# df = df[df.columns[df.isna().mean() < 0.5]]

# Rename columns
dictionary_nhanes = pd.read_csv(dictionary_path)
mapping_dict = dict(zip(dictionary_nhanes['variable_codename_use'], dictionary_nhanes['variable_description_use']))
mapping_dict.update({'RIDAGEYR': 'age', 'RIAGENDR': 'gender'})
df = df.rename(columns=mapping_dict)

# Fix gender data
df['gender'] = df['gender'].astype(str).replace({'2': 'Female', '1': 'Male'})
# df.to_csv('data_inner/raw_nhanes_data.csv', index=False)
not_replicated_columns = df.columns[~df.columns.str.contains('replicate')]
df = df[not_replicated_columns]
df = df[df['age']>=18]
df


chemicals_clean.csv
chemicals_unclean.csv
comments_clean.csv
comments_unclean.csv
demographics_clean.csv
demographics_unclean.csv
dictionary_drug_codes.csv
dictionary_harmonized_categories.csv
dictionary_nhanes.csv
dietary_clean.csv
dietary_unclean.csv
example_0 - merge_datasets_together.Rmd
example_1 - account_for_nhanes_design.Rmd
example_2 - calculate_summary_statistics.Rmd
example_3 - run_multiple_regressions.Rmd
m - nhanes_1988_2018.R
medications_clean.csv
medications_unclean.csv
mortality_clean.csv
mortality_unclean.csv
nhanes_inconsistencies_documentation.xlsx
occupation_clean.csv
occupations_unclean.csv
questionnaire_clean.csv
questionnaire_unclean.csv
response_clean.csv
response_unclean.csv
w - nhanes_1988_2018.RData
weights_clean.csv
weights_unclean.csv


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_79545/451049769.py:26: DtypeWarning: Columns (828,829,836,837,846,847,947,948,980,981) have mixed types. Specify dtype option on import or set low_memory=False.
  response = pd.read_csv(response_path)


Respondent sequence number  \
0                              3.0   
1                              4.0   
2                              9.0   
3                             10.0   
4                             11.0   
...                            ...   
121738                    102949.0   
121740                    102952.0   
121741                    102953.0   
121742                    102954.0   
121744                    102956.0   

       Respondent sequence number that includes an identifier for NHANES III and NHANES continuous  \
0                                                     I-3                                            
1                                                     I-4                                            
2                                                     I-9                                            
3                                                    I-10                                            
4                                                    I-11                                            
...                                                   ...                                            
121738                                           C-102949                                            
121740                                           C-102952                                            
121741                                           C-102953                                            
121742                                           C-102954                                            
121744                                           C-102956                                            

        gender  age  Unnamed: 0_x  SDDSRVYR_x  \
0         Male   21          2597          -1   
1       Female   32          2598          -1   
2       Female   48             1          -1   
3         Male   35          2600          -1   
4         Male   48          2601          -1   
...        ...  ...           ...         ...   
121738    Male   33        121739          10   
121740  Female   70        121741          10   
121741    Male   42        121742          10   
121742  Female   41        121743          10   
121744    Male   38        121745          10   

        Serum cis-Beta carotene (ug/dL)  Acrylamide (pmoL/g Hb)  \
0                                   NaN                     NaN   
1                                   NaN                     NaN   
2                                   NaN                     NaN   
3                                   NaN                     NaN   
4                                   NaN                     NaN   
...                                 ...                     ...   
121738                            0.495                     NaN   
121740                            4.790                     NaN   
121741                            0.495                     NaN   
121742                            1.690                     NaN   
121744                            0.495                     NaN   

        Glycideamide (pmol/g Hb)  Decanaldehyde (ng/mL)  \
0                            NaN                    NaN   
1                            NaN                    NaN   
2                            NaN                    NaN   
3                            NaN                    NaN   
4                            NaN                    NaN   
...                          ...                    ...   
121738                       NaN                    NaN   
121740                       NaN                    NaN   
121741                       NaN                    NaN   
121742                       NaN                    NaN   
121744                       NaN                    NaN   

        Propanaldehyde (ng/mL)  Butyraldehyde (ng/mL)  Pentanaldehyde (ng/mL)  \
0                          NaN                    NaN                     NaN   
1                          NaN                    NaN        

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

feature = 'Peak expiratory flow, largest value (ml)'
dff = df[[feature, 'gender', 'age']].dropna()

# Computing percentiles within each age group for FEV
# Here we prepare a DataFrame where each column represents an age and each row represents a percentile (0-100)

# Getting unique ages
unique_ages = dff['age'].unique()

# Creating an empty DataFrame for percentiles
percentile_df = pd.DataFrame(index=range(1, 101))  # 1-100 percentiles

for age in unique_ages:
    age_specific_fev = dff[dff['age'] == age][feature]
    percentiles = np.percentile(age_specific_fev, range(1, 101))
    percentile_df[age] = percentiles

# Sorting columns by age for easier interpretation
percentile_df = percentile_df[sorted(percentile_df.columns)]


# Selecting a few age groups for visualization to avoid clutter
selected_ages = [20, 30, 40, 50, 60]

# Creating a new DataFrame for selected ages
selected_percentile_df = percentile_df[selected_ages]

# Melting the DataFrame for easier plotting with seaborn
melted_df = selected_percentile_df.reset_index().melt(id_vars='index', value_vars=selected_ages)
melted_df.rename(columns={'index': 'Percentile', 'variable': 'Age', 'value': 'FEV'}, inplace=True)

# Plotting
plt.figure(figsize=(12, 8))
sns.lineplot(data=melted_df, x='Percentile', y='FEV', hue='Age', marker='o')
plt.title('FEV Percentiles Across Different Ages')
plt.xlabel('Percentile')
plt.ylabel('FEV (ml)')
plt.grid(True)
plt.legend(title='Age')
plt.show()

In [33]:
feature


'Peak expiratory flow, largest value (ml)'

In [49]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('../ai-curator-416105-3cdafdc34fc8.json', scope)

# Authenticate and open the spreadsheet
gc = gspread.authorize(credentials)
spreadsheet = gc.open(f'{feature}')

# Select the first sheet
worksheet = spreadsheet.get_worksheet(0)

# Update the sheet with DataFrame values
worksheet.update([percentile_df.columns.values.tolist()] + percentile_df.values.tolist())


{'spreadsheetId': '1gYvd7i4Xtwmgl7Tb0iKACM2j-Q6eN0qL_c3i7ABH2uk',
 'updatedRange': 'Sheet1!A1:BU101',
 'updatedRows': 101,
 'updatedColumns': 73,
 'updatedCells': 7373}

In [50]:
credentials

In [47]:
worksheet.

{'spreadsheetId': '1gYvd7i4Xtwmgl7Tb0iKACM2j-Q6eN0qL_c3i7ABH2uk',
 'replies': [{}]}

In [38]:
gc.open_by_key('1gYvd7i4Xtwmgl7Tb0iKACM2j-Q6eN0qL_c3i7ABH2uk')

<Spreadsheet 'Peak expiratory flow, largest value (ml)' id:1gYvd7i4Xtwmgl7Tb0iKACM2j-Q6eN0qL_c3i7ABH2uk>

In [41]:
f'AIKA/bioage_percentiles/Peak expiratory flow, largest value (ml)'


'/AIKA/bioage_percentiles/Peak expiratory flow, largest value (ml)'